In [ ]:
!pip install filterpy deepface matplotlib pandas opencv-python keras==2.12 batch-face keras_applications keras_preprocessing keras-vggface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cu

In [ ]:
!git clone https://github.com/ElenaRyumina/EMO-AffectNetModel/

Cloning into 'EMO-AffectNetModel'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 187 (delta 14), reused 14 (delta 5), pack-reused 159
Receiving objects: 100% (187/187), 200.84 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
%cd EMO-AffectNetModel

/content/EMO-AffectNetModel


In [ ]:
!python run.py --path_video /content/videos/ --path_save /content/reports/ --path_FE_model /content/models/weights_0_66_37.h5 --path_LSTM_model /content/models/for_RAVDESS.h5

1/1
Downloading: "https://github.com/elliottzheng/face-detection/releases/download/0.0.1/mobilenet0.25_Final.pth" to /root/.cache/torch/hub/checkpoints/mobilenet0.25_Final.pth
100% 1.71M/1.71M [00:00<00:00, 46.9MB/s]
Name video:  interview_zel.mp4
Number total of frames:  642
FPS:  25.0
Video duration: 25.68 s
Frame width: 1280
Frame height: 720
Report saved in:  /content/reports/interview_zel.csv
Lead time: 83.79 s



In [ ]:
!zip -r /content/report.zip /content/reports

  adding: content/reports/ (stored 0%)
  adding: content/reports/interview_zel.csv (deflated 93%)


In [ ]:
from google.colab import files
files.download("/content/report.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import os
from filterpy.kalman import KalmanFilter
import numpy as np
from deepface import DeepFace
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Function to split video into frames and save them
def split_video_to_frames(video_path, frames_dir):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    frames_path = os.path.join(frames_dir, video_name)
    if not os.path.exists(frames_path):
        os.makedirs(frames_path)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = os.path.join(frames_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()
    print(f"Saved {frame_count} frames to {frames_path}")

# Function to initialize a Kalman filter for tracking
def initialize_kalman():
    kf = KalmanFilter(dim_x=7, dim_z=4)
    dt = 1.0  # time interval

    # State transition matrix
    kf.F = np.array([[1, 0, 0, 0, dt, 0, 0],
                     [0, 1, 0, 0, 0, dt, 0],
                     [0, 0, 1, 0, 0, 0, dt],
                     [0, 0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 1]])

    # Measurement function
    kf.H = np.array([[1, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0]])

    # Covariance matrix
    kf.P *= 1000.0
    kf.P[4:, 4:] *= 1000.0

    # Process noise
    kf.Q *= 0.1

    # Measurement noise
    kf.R = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

    return kf

# Function to detect faces in a frame using DeepFace with RetinaFace as the backend
def detect_faces(frame):
    faces = DeepFace.extract_faces(frame, detector_backend='retinaface', enforce_detection=False)
    return faces

def draw_bounding_boxes(frame, faces, ids, dominant_emotion):
    for (x, y, w, h), face_id, emotion in zip(faces, ids, dominant_emotion):
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f"ID: {face_id}", (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.putText(frame, f"{emotion}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Function to calculate cosine similarity between two embeddings
def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Function to update the embedding by weighted average
def update_embedding(old_embedding, new_embedding, alpha=0.5):
    return alpha * np.array(old_embedding) + (1 - alpha) * np.array(new_embedding)

# Function to process frames and plot graphs
def process_frames(frames_dir, graph_dir, video_name, emotion_csv):
    frames_path = os.path.join(frames_dir, video_name)
    graph_path = os.path.join(graph_dir, video_name)
    if not os.path.exists(graph_path):
        os.makedirs(graph_path)

    # Load emotion predictions from CSV
    df_emotions = pd.read_csv(emotion_csv)

    # Add dominant_emotion column to the DataFrame
    emotion_columns = ['Neutral', 'Happiness', 'Sadness', 'Fear', 'Disgust', 'Anger', 'Surprise']
    df_emotions['dominant_emotion'] = df_emotions[emotion_columns].idxmax(axis=1)

    emotion_colors = {
        'Happiness': 'green',
        'Neutral': 'orange',
        'Surprise': 'orange',
        'Sadness': 'red',
        'Anger': 'red',
        'Fear': 'red',
        'Disgust': 'red'
    }

    frame_files = sorted([f for f in os.listdir(frames_path) if f.endswith('.jpg')])
    total_frames = len(frame_files)

    # Initialize a dictionary to keep track of cumulative emotion segments
    cumulative_emotion_frames = {emotion: [] for emotion in emotion_colors}

    for frame_count, frame_file in enumerate(frame_files):
        frame_path = os.path.join(frames_path, frame_file)
        frame = cv2.imread(frame_path)

        # Find emotion data up to the current frame
        current_emotion_data = df_emotions[df_emotions['frame'] <= frame_count]

        # Update cumulative emotion segments
        for index, row in current_emotion_data.iterrows():
            emotion = row['dominant_emotion']
            start_frame = row['frame']
            end_frame = start_frame + 1
            cumulative_emotion_frames[emotion].append((start_frame, end_frame))

        # Plot the emotions graph for the current frame
        fig, ax = plt.subplots(figsize=(10, 6))

        # Plot the Gantt chart for cumulative emotions up to the current frame
        for emotion, segments in cumulative_emotion_frames.items():
            for start_frame, end_frame in segments:
                ax.barh(emotion, end_frame - start_frame, left=start_frame, color=emotion_colors[emotion], height=0.5)

        # Customize the plot
        ax.set_xlabel('Frames')
        ax.set_ylabel('Emotions')
        ax.set_title(f'Emotion Detection Over Frames (Up to Frame {frame_count})')
        ax.invert_yaxis()  # Invert y axis to keep the same order as the provided image
        ax.set_yticks(range(len(emotion_colors)))
        ax.set_yticklabels(list(emotion_colors.keys()))
        ax.set_xlim(0, total_frames)  # Set x-axis limit to total number of frames

        # Save the graph plot as an image
        graph_frame_filename = os.path.join(graph_path, f"emotions_graph_frame_{frame_count}.jpg")
        fig.savefig(graph_frame_filename)
        plt.close(fig)

        print(f"Processed frame {frame_count}")

    print(f"All frames processed and saved.")

# Path to your video file
video_path = '/content/videos/input.mp4'

# Directory to store frames
frames_dir = '/content/frames'

# Directory to store graphs
graph_dir = '/content/graphs'

# Path to your emotion CSV file
emotion_csv = '/content/reports/output.csv'

# Split the video into frames
split_video_to_frames(video_path, frames_dir)

# Process the frames and generate graphs
video_name = os.path.splitext(os.path.basename(video_path))[0]
process_frames(frames_dir, graph_dir, video_name, emotion_csv)

Saved 1619 frames to /content/frames/vid_trimmed
Processed frame 0
Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed f

In [ ]:
!zip -r /content/drive/MyDrive/sentiment/output.zip /content/graphs